# Lesson 02: Image Processing Without OpenCV

Before relying on high-level libraries, it's important to understand that **images are just NumPy arrays**.  
In this notebook we perform image operations using **only NumPy** — no OpenCV functions.

This builds intuition for what OpenCV is doing under the hood.

## Topics:
- Images as arrays
- Manual grayscale conversion
- Manual brightness / contrast adjustment
- Manual cropping and flipping
- Manual channel manipulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image   # only used for loading — no processing

# Load image as numpy array (RGB) using PIL
pil_img = Image.open('sealion_hero.png').convert('RGB')
img = np.array(pil_img, dtype=np.uint8)

print(f'Array shape: {img.shape}')    # (H, W, 3)
print(f'Data type:   {img.dtype}')    # uint8
print(f'Min / Max:   {img.min()} / {img.max()}')

plt.figure(figsize=(6, 4))
plt.imshow(img)
plt.title('Original Image (loaded as numpy array)')
plt.axis('off')
plt.show()

## 1. Grayscale Conversion — Manual

The standard luminance formula (ITU-R BT.601):
$$
\text{Gray} = 0.299 \cdot R + 0.587 \cdot G + 0.114 \cdot B
$$
Green gets the highest weight because the human eye is most sensitive to it.

In [ ]:
# Manual grayscale using the luminance formula
R = img[:, :, 0].astype(np.float32)
G = img[:, :, 1].astype(np.float32)
B = img[:, :, 2].astype(np.float32)

gray_manual = (0.299 * R + 0.587 * G + 0.114 * B).astype(np.uint8)

# Compare: simple average vs proper formula
gray_avg = ((R + G + B) / 3).astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(13, 4))
axes[0].imshow(img);                      axes[0].set_title('Original RGB');   axes[0].axis('off')
axes[1].imshow(gray_avg, cmap='gray');    axes[1].set_title('Simple Average'); axes[1].axis('off')
axes[2].imshow(gray_manual, cmap='gray'); axes[2].set_title('Luminance Formula (0.299R + 0.587G + 0.114B)'); axes[2].axis('off')
plt.tight_layout()
plt.show()

## 2. Brightness Adjustment — Manual

Adding or subtracting a constant from each pixel.  
We must **clip** values to stay in [0, 255] to avoid overflow.

In [ ]:
def adjust_brightness(image, delta):
    """Add delta to all pixels and clip to [0, 255]."""
    result = image.astype(np.int16) + delta
    return np.clip(result, 0, 255).astype(np.uint8)

bright = adjust_brightness(img, +80)
dark   = adjust_brightness(img, -80)

fig, axes = plt.subplots(1, 3, figsize=(13, 4))
axes[0].imshow(img);    axes[0].set_title('Original');   axes[0].axis('off')
axes[1].imshow(bright); axes[1].set_title('+80 Brightness'); axes[1].axis('off')
axes[2].imshow(dark);   axes[2].set_title('−80 Brightness'); axes[2].axis('off')
plt.tight_layout()
plt.show()

## 3. Contrast Adjustment — Manual

Multiplying pixel values by a factor scales the contrast:
- Factor > 1.0 → increases contrast (darks darker, brights brighter)
- Factor < 1.0 → decreases contrast (range compresses)

In [ ]:
def adjust_contrast(image, factor):
    """Multiply all pixels by factor and clip to [0, 255]."""
    result = image.astype(np.float32) * factor
    return np.clip(result, 0, 255).astype(np.uint8)

low_contrast  = adjust_contrast(img, 0.5)
high_contrast = adjust_contrast(img, 1.8)

fig, axes = plt.subplots(1, 3, figsize=(13, 4))
axes[0].imshow(img);           axes[0].set_title('Original');        axes[0].axis('off')
axes[1].imshow(low_contrast);  axes[1].set_title('Low contrast ×0.5'); axes[1].axis('off')
axes[2].imshow(high_contrast); axes[2].set_title('High contrast ×1.8'); axes[2].axis('off')
plt.tight_layout()
plt.show()

## 4. Cropping and Flipping — Manual (Pure NumPy)

In [ ]:
h, w = img.shape[:2]

# Crop: numpy slice [y_start:y_end, x_start:x_end]
crop = img[h//4 : 3*h//4, w//4 : 3*w//4]

# Horizontal flip: reverse columns
flip_h = img[:, ::-1, :]

# Vertical flip: reverse rows
flip_v = img[::-1, :, :]

# Both flips
flip_both = img[::-1, ::-1, :]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for ax, (im, t) in zip(axes, [
    (crop,      'Cropped (center)'),
    (flip_h,    'Flip horizontal'),
    (flip_v,    'Flip vertical'),
    (flip_both, 'Flip both'),
]):
    ax.imshow(im); ax.set_title(t); ax.axis('off')
plt.tight_layout()
plt.show()

## 5. Channel Manipulation — Manual

Images have 3 color channels (R, G, B).  
We can isolate or zero out individual channels to see their contribution.

In [ ]:
# Isolate each channel (keep one, set others to 0)
red_only   = img.copy(); red_only[:, :, 1] = 0;   red_only[:, :, 2] = 0
green_only = img.copy(); green_only[:, :, 0] = 0;  green_only[:, :, 2] = 0
blue_only  = img.copy(); blue_only[:, :, 0] = 0;   blue_only[:, :, 1] = 0

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
axes[0].imshow(img);        axes[0].set_title('Original');    axes[0].axis('off')
axes[1].imshow(red_only);   axes[1].set_title('Red channel'); axes[1].axis('off')
axes[2].imshow(green_only); axes[2].set_title('Green channel'); axes[2].axis('off')
axes[3].imshow(blue_only);  axes[3].set_title('Blue channel'); axes[3].axis('off')
plt.tight_layout()
plt.show()

## 6. Image Negative (Inversion)

The **negative** of an image flips all pixel intensities:  
$$\text{neg}(x, y) = 255 - I(x, y)$$

In [ ]:
negative = 255 - img

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(img);      axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(negative); axes[1].set_title('Negative (255 - img)'); axes[1].axis('off')
plt.tight_layout()
plt.show()

## 7. Float32 vs uint8 — Why It Matters

When doing math on uint8 arrays, overflow wraps around (255 + 1 = 0)!  
Always convert to float32 for calculations, then clip and convert back.

In [ ]:
# Dangerous: uint8 overflow
overflow_bad = img + np.uint8(100)   # wraps around!

# Safe: use float32 + clip
overflow_safe = np.clip(img.astype(np.float32) + 100, 0, 255).astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(13, 4))
axes[0].imshow(img);            axes[0].set_title('Original');                  axes[0].axis('off')
axes[1].imshow(overflow_bad);   axes[1].set_title('+100 (uint8 wrap — WRONG)'); axes[1].axis('off')
axes[2].imshow(overflow_safe);  axes[2].set_title('+100 (float+clip — CORRECT)'); axes[2].axis('off')
plt.tight_layout()
plt.show()

## Summary

- Images are **NumPy arrays** — shape `(H, W, 3)` for color, `(H, W)` for grayscale
- Operations are just **array math**: slicing = crop, `[::-1]` = flip, etc.
- Always work in **float32** when computing, then convert back to **uint8** with `np.clip(..., 0, 255)`
- OpenCV functions are convenience wrappers around these exact same operations